In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import re
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk.tokenize.punkt
from gensim.utils import lemmatize
from nltk.stem.snowball import SnowballStemmer
#import Pattern
from textblob import TextBlob,Word
from sklearn.cluster import KMeans
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

In [ ]:
def splitter(dataset):
    #dataset=data.copy(deep=True)
    dataset['SplitText']=''
    index=0
    for query_raw in dataset['Problem Statement']:
        separator_found = False
        if (type(query_raw) != str): #check for nan values
            #print(query)
            continue
        query=re.sub('[^\w\d\s]+',' ',query_raw)
        query=re.sub('\s+',' ',query)
        #print(query)
        words_in_query = query.split()
        for word_index in range(1, len(words_in_query)):
            stemmed_lower_word = snowball.stem(words_in_query[word_index].lower())
            #print("{}-------------{}".format(words_in_query[word_index], stemmed_lower_word))
            if (stemmed_lower_word in distinct_stemmed_separators):
                #print('YES')
                #found the separator word, now seperate the raw query
                #general_info.append(' '.join(words_in_query[0:word_index]))
                text=' '.join(words_in_query[word_index:len(words_in_query)])
                dataset.at[index,'SplitText']=text
                #dataset.at[i,'SplitText']='asdfg'
                #print(dataset.loc[index,'SplitText'])
                separator_found = True
                index+=1
                break
        if (separator_found == False):
            #false_count+=1
            #if there is totally no separator word in the query, add in the whole query
            #separator_found_false_count += 1
            #print(query)
            #print("------------------------------------------------------------------------------------------\n\n")
            #general_info.append(" ")
            #actual_query.append(query)
            dataset.at[index,'SplitText']=query
            index+=1
    return dataset


## Need to install averaged_perceptron_tagger in a folder named 'taggers' first

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.VERB)

NON_ALPHANUM=re.compile(r'[\W]')
NON_ASCII=re.compile(r'[^a-z0-1\s]')

## CREATE LEMMATIZER OBJECT
lemmatizer=WordNetLemmatizer()

def normalize_text(texts):
    lower=texts.lower()
    no_punctuation=NON_ALPHANUM.sub(r' ',lower)
    no_non_ascii=NON_ASCII.sub(r'',no_punctuation)
    return no_non_ascii

def clean_text(dataset):
    dataset['Problem Statement Filtered']=dataset['SplitText'].apply(normalize_text)
    dataset['Problem Statement Filtered']=dataset['Problem Statement Filtered'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in x.split() if len(w)>2
                                                                             and not lemmatizer.lemmatize(w,get_wordnet_pos(w)) in all_stopwords]))
    return dataset
                        

def get_top_n_words(corpus, n=10):
  vec = CountVectorizer(stop_words=all_stopwords,ngram_range=(1,2)).fit(corpus)
  bag_of_words = vec.transform(corpus)
  sum_words = bag_of_words.sum(axis=0) 
  words_freq = [(word, sum_words[0, idx]) for word, idx in   vec.vocabulary_.items()]
  words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
  return words_freq[:n]

In [ ]:
nltk.data.path.append("C:/Users/ESG-JT5190/Documents/Python Scripts")
os.getcwd()
os.chdir(r'C:\Users\ESG-JT5190\Documents\Project Code Repo\data')
os.getcwd()

In [ ]:
data=pd.read_csv('./processed/data_consolidated_20200629.csv',engine='python')

In [ ]:
data.loc[1,'Problem Statement']

In [ ]:
data20=data[data.year==20]
print('Number of rows and columns for 2020 dataset is {}'.format(data20.shape))

data19=data[data.year==19]
print('Number of rows and columns for 2019 dataset is {}'.format(data19.shape))

In [ ]:
data20.dtypes

In [ ]:
Others20=data20[(data20.category_digital==False) & (data20.category_manpower==False)]
print('Number of rows and columns for 2020 Others dataset is {}'.format(Others20.shape))
Digital20=data20[data20.category_digital==True]
print('Number of rows and columns for 2020 Digital dataset is {}'.format(Digital20.shape))
Manpower20=data20[data20.category_manpower==True]
print('Number of rows and columns for 2020 Manpower dataset is {}'.format(Manpower20.shape))

In [ ]:
## Use 2 months for Jan and Feb
Others20=Others20[(Others20['month']=='Jan') | (Others20['month']=='Feb')]
print('Number of rows and columns for 2020 Others (Jan and Feb) dataset is {}'.format(Others20.shape))
Digital20=Digital20[(Digital20['month']=='Jan') | (Digital20['month']=='Feb')]
print('Number of rows and columns for 2020 Digital (Jan and Feb) dataset is {}'.format(Digital20.shape))
Manpower20=Manpower20[(Manpower20['month']=='Jan') | (Manpower20['month']=='Feb')]
print('Number of rows and columns for 2020 Manpower (Jan and Feb) dataset is {}'.format(Manpower20.shape))

Others20=Others20.reset_index(drop=True)
Digital20=Digital20.reset_index(drop=True)
Manpower20=Manpower20.reset_index(drop=True)

In [ ]:
### USE LIST OF STOPWORDS FROM R LIBRARY COMBINED WITH CUSTOM STOPWORDS AS I FELT THAT THE STOPWORDS PROVIDED BY PYTHON
### LIBRARIES ARE NOT ENOUGH. WORDS SUCH AS 'LIKE' ARE NOT STOPWORDS IN PYTHON LIBRARIES
all_stopwords=open('./raw/all_stopwords.txt','r')
all_stopwords=all_stopwords.read().split('\n')
len(all_stopwords)

In [ ]:
snowball = SnowballStemmer("english")

## READ IN SEPARATOR WORDS
raw_separators = open(r'.\raw\separators.txt','r')
raw_separators=raw_separators.read().split('\n')
#additional_stopwords
print("Length: {}\nList: {}\n".format(len(raw_separators), sorted(raw_separators)))


stemmed_separators = []

for separator in raw_separators:
    stemmed_separators.append(snowball.stem(separator))

print("Length: {}\nList: {}\n".format(len(stemmed_separators), sorted(stemmed_separators)))

distinct_stemmed_separators = sorted(list(set(stemmed_separators)))

print("Length: {}\nList: {}\n".format(len(distinct_stemmed_separators), distinct_stemmed_separators))

In [ ]:
## DO OTHERS SUBSET FIRST
## SPLIT THE TEXT FOR OTHERS SUBSET
Others20=splitter(Others20)

In [ ]:
Others20['SplitText']

In [ ]:
## CLEAN THE TEXT
Others20=clean_text(Others20)

In [ ]:
Others20.tail(5)

In [ ]:
direc=os.path.dirname(os.getcwd())
print(direc)
glove_filename=os.path.join(direc,'data','raw','glove.6B.100d.txt')
print(glove_filename)
direc=os.path.dirname(os.getcwd())
tmp_filename=os.path.join(direc,'data','raw','tmp_file.txt')
print(tmp_filename)

In [ ]:
### READ IN WORD EMBEDDINGS FILE
glove_file=datapath(glove_filename)
tmp_file=get_tmpfile(tmp_filename)

_=glove2word2vec(glove_file,tmp_file)

word2vec=KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
word='iraq'
print('Word: {}'.format(word))
print('First 20 values of embedding:\n{}'.format(word2vec[word][:20]))

In [ ]:
print(word2vec.most_similar(positive=['woman', 'king'], negative=['man'], topn=3))
print(word2vec.most_similar(positive=['tennis', 'ronaldo'], negative=['soccer'], topn=3))

In [ ]:
class WordVecVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = 100   
        
    def fit(self, X, y):
        return self    
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in texts.split() if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for texts in X
        ])#representing each statement by the mean of word embeddings for the words used in the statement.


wtv_vect = WordVecVectorizer(word2vec)
X_train_wtv_others = wtv_vect.transform(Others20['Problem Statement Filtered'])
print(X_train_wtv_others.shape)


In [ ]:
clusters_number=5

km = KMeans(
    n_clusters=clusters_number, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=1000
)
y_km_others = km.fit_predict(X_train_wtv_others)
clusters_df_others = pd.DataFrame({'Problem Statement Filtered' :Others20['Problem Statement Filtered'], 'topic_cluster' :y_km })

In [ ]:
clusters_df_others[clusters_df_others['topic_cluster']==4]

In [ ]:
for i in range(0,clusters_number):
    globals()['Cluster_%s_Others' %i]=clusters_df_others[clusters_df_others['topic_cluster']==i]
    print('Cluster',i,'has dimensions',globals()['Cluster_%s_Others' %i].shape)

In [ ]:
for k in range(0,clusters_number):
    #print(type(i))
    words = []
    word_values = []
    for i,j in get_top_n_words(globals()['Cluster_%s_Others' %k]['Problem Statement Filtered'],10):
      words.append(i)
      word_values.append(j)
    print('Top 10 words for CLuster',k,':',", ".join(words),'\n')

In [ ]:
### DO DIGITAL NOW
## SPLIT THE TEXT FOR Digital SUBSET
Digital20=splitter(Digital20)

In [ ]:
Digital20['SplitText']

In [ ]:
## CLEAN THE TEXT
Digital20=clean_text(Digital20)

In [ ]:
X_train_wtv_digi = wtv_vect.transform(Digital20['Problem Statement Filtered'])
print(X_train_wtv_digi.shape)

In [ ]:
y_km_digi = km.fit_predict(X_train_wtv_digi)
clusters_df_digi = pd.DataFrame({'Problem Statement Filtered' :Digital20['Problem Statement Filtered'], 'topic_cluster' :y_km_digi })

In [ ]:
for i in range(0,clusters_number):
    globals()['Cluster_%s_digi' %i]=clusters_df_digi[clusters_df_digi['topic_cluster']==i]
    print('Cluster',i,'has dimensions',globals()['Cluster_%s_digi' %i].shape)

In [ ]:
for k in range(0,clusters_number):
    #print(type(i))
    words = []
    word_values = []
    for i,j in get_top_n_words(globals()['Cluster_%s_digi' %k]['Problem Statement Filtered'],10):
      words.append(i)
      word_values.append(j)
    print('Top 10 words for Cluster',k,':',", ".join(words),'\n')

In [ ]:
### DO MANPOWER NOW
## SPLIT THE TEXT FOR Digital SUBSET
Manpower20=splitter(Manpower20)

In [ ]:
## CLEAN THE TEXT
Manpower20=clean_text(Manpower20)

In [ ]:
X_train_wtv_man = wtv_vect.transform(Manpower20['Problem Statement Filtered'])
print(X_train_wtv_man.shape)

In [ ]:
y_km_man = km.fit_predict(X_train_wtv_man)
clusters_df_man = pd.DataFrame({'Problem Statement':Manpower20['Problem Statement']
    ,'SplitText':Manpower20['SplitText'],
    'Problem Statement Filtered' :Manpower20['Problem Statement Filtered'], 'topic_cluster' :y_km_man })

In [ ]:
for i in range(0,clusters_number):
    globals()['Cluster_%s_man' %i]=clusters_df_man[clusters_df_man['topic_cluster']==i]
    print('Cluster',i,'has dimensions',globals()['Cluster_%s_man' %i].shape)

In [ ]:
for k in range(0,clusters_number):
    #print(type(i))
    words = []
    word_values = []
    for i,j in get_top_n_words(globals()['Cluster_%s_man' %k]['Problem Statement Filtered'],10):
      words.append(i)
      word_values.append(j)
    print('Top 10 words for Cluster',k,':',", ".join(words),'\n')